In [1]:
import nest_asyncio

import autogen
from autogen.agents.experimental import WebSurferAgent

nest_asyncio.apply()

In [2]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["gpt-4o-mini"]},
)
llm_config = {"config_list": config_list}

In [ ]:
from autogen import ConversableAgent, GroupChat, GroupChatManager, UserProxyAgent

# Swarm Agents, similar to ConversableAgent, but with functions and hand offs (specified later)
initial_analyser = ConversableAgent(
    name="InitialAnalysisAgent",
    system_message="""You are an expert at breaking down complex questions into smaller, focused subquestions.
Your task is to take any question provided and divide it into clear, actionable subquestions that can be individually answered.
Ensure the subquestions are logical, non-redundant, and cover all key aspects of the original question.
Avoid providing answers or interpretations—focus solely on decomposition.

Do NOT forward the initial task without breaking it down into subquestions!!!
""",
    llm_config=llm_config,
)

websurfer = WebSurferAgent(
    llm_config=llm_config,
    name="WebSurferAgent",
    system_message="""You are a web surfer agent responsible for gathering information from the web to answer the subquestions provided.
Always perform searches one subquestion at a time. You are allowed to perform a new search only after you receive a new subquestion.
""",
    web_tool="crawl4ai",
)

summarizer = ConversableAgent(
    name="SummarizerAgent",
    system_message="""You are a summarizer agent responsible for gathering and synthesizing answers to subquestions to provide a complete and concise response to the original question.
If the gathered information is insufficient to answer the original question, request further analysis from the initial analysis agent by specifying what is missing.
Otherwise, combine the information into a clear and comprehensive summary that addresses the original question.""",
    llm_config=llm_config,
)

for tool in websurfer.tools:
    websurfer.register_for_execution()(tool)


user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
)

groupchat = GroupChat(
    agents=[user_proxy, initial_analyser, websurfer, summarizer],
    messages=[],
    max_round=20,
)

manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
    # is_termination_msg=lambda x: "GROUPCHAT_TERMINATE" in x.get("content", ""),
)

user_proxy.initiate_chat(
    manager,
    message="Which are the best cryptocurrencies to invest in? Do the web research first and make som conclusions",  # Initial message
)